In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from matplotlib import animation, rc
from IPython.display import HTML
from numba import jit

In [ ]:
angle_range = np.linspace(0,2*np.pi,1000)
line_range = np.linspace(-1.0, 1.0, 1000)
circle_x = np.cos(angle_range)
circle_y = np.sin(angle_range)
circle_z = np.zeros_like(angle_range)

circle_3d = np.vstack((circle_x, circle_y, circle_z))
circle_3d.shape

In [ ]:
fig = plt.figure(figsize=(6,6))# (figsize=(7.5,6))

# w = 1.0

# p = (1. - w) / 2.

# ax = fig.add_axes([p, p, w, w], projection='3d')
ax = fig.add_subplot(111, projection='3d')
ax.dist=6
# ex = 0.35

# fig.subplots_adjust(left=-ex, bottom=-ex, right=1. + ex, top=1. + ex, wspace=None, hspace=None)
ax.set_aspect('equal')
ax.set_axis_off()
plt.plot(circle_x, circle_y, circle_z, color='gray')
plt.plot(circle_z, circle_x, circle_y, color='gray')
plt.plot(circle_y, circle_z, circle_x, color='gray')
plt.plot(line_range, circle_z, circle_z, color='gray')
plt.plot(circle_z, line_range, circle_z, color='gray')
plt.plot(circle_z, circle_z, line_range, color='gray')


plt.savefig("test.png")
plt.show()

In [ ]:
def init_bloch_sphere():
    fig = plt.figure(figsize=(6,6))# (figsize=(7.5,6))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_aspect('equal')
    ax.set_axis_off()
    plt.plot(circle_x, circle_y, circle_z, color='gray')
    plt.plot(circle_z, circle_x, circle_y, color='red')
    plt.plot(circle_y, circle_z, circle_x, color='gray')
    plt.plot(line_range, circle_z, circle_z, color='gray')
    plt.plot(circle_z, line_range, circle_z, color='gray')
    plt.plot(circle_z, circle_z, line_range, color='gray')
    
    return fig, ax

def spher_to_cart(theta, phi, r=1.0):
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    return np.array([x, y, z])

def plot_vector(x, y, z, **kw):
    line = plt.plot([0.0, x], [0.0, y], [0.0, z], **kw)
    dot = plt.plot([x], [y], [z], marker='o', zorder=1000, **kw)
    return line, dot
    
def plot_vector_sph(theta, phi, **kw):
    r = kw.pop('r', 1.0)
    plot_vector(*spher_to_cart(theta, phi, r), **kw)

In [ ]:
fig, ax = init_bloch_sphere()
plot_vector_sph(0.0,-np.pi/2)

In [ ]:
fig, ax = init_bloch_sphere()

(line,) = plt.plot([], [], [], color='C1', animated=True, linewidth=5)
(dot,) = plt.plot([], [], [], marker='o', zorder=1000, color='C1')

def general_update(theta, phi):
    x, y, z = spher_to_cart(theta, phi)
    line.set_data([0.0, x], [0.0, y])
    line.set_3d_properties([0.0, z])
    dot.set_data([x], [y])
    dot.set_3d_properties([z])
    
ani = animation.FuncAnimation(fig, lambda t: general_update(t, -np.pi/2), frames=np.linspace(0, np.pi, 100), interval=20)

ani.save("simple-pi.mp4")

from IPython.display import HTML
HTML(ani.to_jshtml())

## Numerical integration of Hamiltonians

In [ ]:
sx = np.array([[0, 1],  [ 1, 0]], dtype=np.complex128)
sy = np.array([[0, -1j],[1j, 0]], dtype=np.complex128)
sz = np.array([[1, 0],  [0, -1]], dtype=np.complex128)
si = np.array([[1, 0],  [ 0, 1]], dtype=np.complex128)
S = np.stack((sx, sy, sz), axis=0)

def x(t):
    return np.zeros_like(t)

def y(t):
    return np.zeros_like(t)

def z(t):
    return np.ones_like(t) * 0.1

def calc_ev_ops(H_pauli, dt):
    N = H_pauli.shape[0]
    # we expect the hamiltonian in pauli-decomposed form [x, y, z]
    
    H_norms = np.linalg.norm(H_pauli, axis=1).reshape(-1, 1)
    angles = H_norms * dt
    
    H_pauli_normed = H_pauli / H_norms
    # is h_norm is 0, we get infinities
    # but in that case the rotation angle h_norm * dt is also zero
    # so we don't really care about the rotation axis and we can set
    # the infinities to zero
    H_pauli_normed[~np.isfinite(H_pauli_normed)] = 0.0 
    R_matrix = np.tensordot(H_pauli_normed, S, axes=([1], [0]))
    
    eyes = np.repeat(np.eye(2)[np.newaxis,:,:], N, axis=0)
    
    ev_ops = np.cos(angles).reshape(-1, 1, 1) * eyes + 1j * np.sin(angles).reshape(-1, 1, 1) * R_matrix
    
    return ev_ops

@jit(nopython=True)
def accumulate_ev_ops(ev_ops):
    N = ev_ops.shape[0]
    
    # complicated way of getting the dtype right for np.eye,
    # apparently, numba doesn't support the dtype parameter for np.eye
    acc_op = np.zeros((2,2), dtype=np.complex128)
    acc_op[:,:] += np.eye(2)
    
    for i in range(N):
        acc_op = np.dot(acc_op, ev_ops[i,:,:])
        
    return acc_op

@jit(nopython=True)
def accumulate_ev_ops_over_state(ev_ops, state):
    N = ev_ops.shape[0]
    result = np.zeros((N+1, 2), dtype=np.complex128)
    
    result[0,:] = state
    for i in range(N):
        result[i+1,:] = np.dot(ev_ops[i,:,:], result[i,:])
        
    return result

In [ ]:
ts = np.linspace(0,np.pi/50, int(1e4))
dt = np.diff(ts)[0]
H_pauli = np.zeros((len(ts), 3))
H_pauli[:,0] = x(ts)
H_pauli[:,1] = y(ts)
H_pauli[:,2] = z(ts)

ev_ops = calc_ev_ops(H_pauli, dt)
acc_ev_op = accumulate_ev_ops(ev_ops)

ev_N = 100
plot_ev_ops = np.repeat(acc_ev_op[np.newaxis, :, :], ev_N, axis=0)

psi_0 = np.ones((2)) / np.sqrt(2)

psi_t = accumulate_ev_ops_over_state(plot_ev_ops, psi_0)

In [ ]:
def state_vector_to_bloch_angles(v):
    orig_shape = v.shape
    v = v.reshape(-1, 2)
    results = np.zeros_like(v, dtype=np.float64)
    
    results[:,0] = 2*np.arccos(np.abs(v[:,0]))
    results[:,1] = np.angle(v[:,1]) - np.angle(v[:,0])
    
    return results.reshape(orig_shape)

## Plot numerical solution

In [ ]:
# plot parameters
axis_width = 5     # line width of the plotted precession axes
qubit_width = 2    # line width of the qubit state vector
qubit_marker = 5   # marker size of the qubit state vector endpoint

fig, ax = init_bloch_sphere()

(qb_line,) = plt.plot([], [], [], color='g', animated=True, linewidth=qubit_width)
(qb_dot,) = plt.plot([], [], [], color='g', animated=True, marker='o', markersize=qubit_marker, linestyle='')

def spher_to_cart_rotating_frame(theta, phi, r=1.0, rot_phase=0.0):
    x = r * np.sin(theta) * np.cos(phi - rot_phase)
    y = r * np.sin(theta) * np.sin(phi - rot_phase)
    z = r * np.cos(theta)
    return np.array([x, y, z])

def update_line(line, c):
    line.set_data([0.0, c[0]], [0.0, c[1]])
    line.set_3d_properties([0.0, c[2]])
    line.set_zorder(-c[1]+100)
    
def update_point(point, c):
    point.set_data(c[0:2])
    point.set_3d_properties(c[2:3])

def update(i):
    psi = psi_t[i,:]
    psi_bloch_s = state_vector_to_bloch_angles(psi)
    psi_bloch_c = spher_to_cart(psi_bloch_s[0], psi_bloch_s[1])
    
    update_line(qb_line, psi_bloch_c)
    update_point(qb_dot, psi_bloch_c)
    
    
ani = animation.FuncAnimation(fig, update, frames=np.arange(100), interval=20)

HTML(ani.to_jshtml())

In [ ]:
%debug

## Lab frame with axes

In [ ]:
# physical parameters
w_0 = 1.0        # S_z qubit frequency
w_R = 0.0        # rotating frame frequency
w_D = 1.0        # drive frequency
H_1 = 0.5        # drive strength (static for now) in units of H_1/w_0 (i.e. w_0 is scaled to 1)

# plot parameters
axis_width = 5     # line width of the plotted precession axes
qubit_width = 2    # line width of the qubit state vector

fig, ax = init_bloch_sphere()

# define plot elements
# static magnetic field precession axis
(w_0_line,) = plt.plot([], [], [], color='k', animated=True, linewidth=axis_width)
(w_0_dot,) = plt.plot([0.0], [0.0], [0.0], color='k', animated=True, marker='o', markersize=2*axis_width, linestyle='', zorder=1000)
# drive field precession axis
(w_D_line,) = plt.plot([], [], [], color='r', animated=True, linewidth=axis_width)
# qubit state vector
(qb_line,) = plt.plot([], [], [], color='g', animated=True, linewidth=qubit_width)

def spher_to_cart_rotating_frame(theta, phi, r=1.0, rot_phase=0.0):
    x = r * np.sin(theta) * np.cos(phi - rot_phase)
    y = r * np.sin(theta) * np.sin(phi - rot_phase)
    z = r * np.cos(theta)
    return np.array([x, y, z])

def update_line(line, c):
    line.set_data([0.0, c[0]], [0.0, c[1]])
    line.set_3d_properties([0.0, c[2]])
    line.set_zorder(-c[1]+100)

def update(t):
    # update the plot for a new frame at time t
    # first we calculate all coordinates in spherical coordinates (to facilitate the rotating frame transformation)
    # [theta, phi, r]
    w_0_angles = [0.0, 0.0, 1.0]
    w_D_angles = [np.pi / 2, w_D * t, H_1]
    
    # apply rotating frame transformation and convert to cartesian coords for plotting
    rotating_frame_phase = w_R * t
    w_0_coords = spher_to_cart_rotating_frame(*w_0_angles, rot_phase=rotating_frame_phase)
    w_D_coords = spher_to_cart_rotating_frame(*w_D_angles, rot_phase=rotating_frame_phase)
    
    # update plot
    update_line(w_0_line, w_0_coords)
    update_line(w_D_line, w_D_coords)
    
ani = animation.FuncAnimation(fig, update, frames=np.linspace(0, 2*np.pi, 100), interval=20)

HTML(ani.to_jshtml())

In [ ]:
A = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8]
])

B = np.array([
    [1, 2, 1, 2],
    [2, 1, 2, 1]
])

np.tensordot(A, B, axes=([1], [1]))

In [ ]:
np.tensordot(A[1,:], B[1,:], axes=([0], [0]))

In [ ]:
np.tensordot?

In [ ]:
np.einsum?

In [ ]:
def multiply_idpauli_matrices(A, B):
    orig_shape = A.shape
    A = A.reshape(-1, 4)
    B = B.reshape(-1, 4)
    N = A.shape[0]

    B_indices = np.array([
        [0, 1, 2, 3],
        [1, 0, 3, 2],
        [2, 3, 0, 1],
        [3, 2, 1, 0]
    ], dtype=np.int)

    B_prefactors = np.array([
        [1,   1,    1,   1],
        [1,   1,   1j, -1j],
        [1, -1j,    1,  1j],
        [1,  1j,  -1j,   1],
    ], dtype=np.complex128)

    C = np.zeros((N, 4), dtype=np.complex128)

    for i in range(4):
        indices = B_indices[i,:]
        prefactors = B_prefactors[i,:]
        C[:,i] = np.einsum('ij,ij->i', A, B[:, indices] * prefactors[np.newaxis, :])

    return C.reshape(orig_shape)

In [ ]:
C = multiply_idpauli_matrices(A, B)

In [ ]:
# Pauli matrices
sx = np.array([[0, 1], [1, 0]], dtype=np.complex128)
sy = np.array([[0, -1j], [1j, 0]], dtype=np.complex128)
sz = np.array([[1, 0], [0, -1]], dtype=np.complex128)
si = np.array([[1, 0], [0, 1]], dtype=np.complex128)
S = np.stack((sx, sy, sz), axis=0)
Sid = np.stack((si, sx, sy, sz), axis=0)

In [ ]:
A_matrix = np.tensordot(A, Sid, axes=([1], [0]))
B_matrix = np.tensordot(B, Sid, axes=([1], [0]))
C_matrix = np.tensordot(C, Sid, axes=([1], [0]))

In [ ]:
np.dot(A_matrix[0,:,:], B_matrix[0,:,:])

In [ ]:
C_matrix[0,:,:]

In [ ]:
F = np.arange(3, dtype=np.complex128).reshape(3)
ang = np.arange(2, dtype=np.complex128)+1

In [ ]:
ang[:,np.newaxis].shape

In [ ]:
def calc_rotation_matrices_pauli(angles, ax_R):
    N = len(angles)
    result = np.zeros((N, 4), dtype=np.complex128)

    result[:,0] = np.cos(angles)
    result[:,1:4] = 1j * np.sin(angles).reshape(-1, 1) * ax_R.reshape(-1, 3)

    return result

In [ ]:
ax = [0., 1., 1.]
ax = np.array(ax)
ax = ax / np.linalg.norm(ax)


H = np.array([
    [0.0, 0.1, 0.0, 1.0]
])


R1 = calc_rotation_matrices_pauli(np.array([0.5]), ax)
R2 = calc_rotation_matrices_pauli(np.array([-0.5]), ax)

In [ ]:
multiply_idpauli_matrices(R1, multiply_idpauli_matrices(H, R2))

In [ ]:
R2

In [ ]:
init_b